In [ ]:
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('drive/My Drive/Telecom_customer churn (100000).csv')

FileNotFoundError: ignored

In [ ]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)

change_mou=num_df['change_mou']
change_rev=num_df['change_rev']
num_df=num_df.drop(['change_mou'], axis=1)
num_df=num_df.drop(['change_rev'], axis=1)

num_df=num_df.drop(['Customer_ID'], axis=1)

churn=num_df['churn']
num_df=num_df.drop(['churn'], axis=1)

#no convertion

num_df=num_df.fillna(conData)

num_df[num_df < 0]=0

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn']
y=churn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train=pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)


In [ ]:
from google.colab import files
import os, sys
file_path = 'drive/My Drive/Colab Notebooks/private-data-generation/'
#sys.path.append(file_path)
sys.path.append(os.path.abspath(file_path))
import sys
sys.path.insert(1,'/content/drive/My Drive/Colab Notebooks/private-data-generation/')

In [ ]:
from models import dp_wgan, pate_gan, ron_gauss

ModuleNotFoundError: ignored

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn import preprocessing
from scipy.special import expit
from models import dp_wgan, pate_gan, ron_gauss
from models.Private_PGM import private_pgm
import argparse
import numpy as np
import pandas as pd
import collections
import os

In [ ]:
class  Class_opt:
    "This is a person class"
    categorical = False
    model='dp-wgan'
    target_epsilon=80
    target_delta=0.8
    downstream_task="classification"
    target_variable='status'
    batch_size=64
    micro_batch_size=8
    clamp_lower=0.01
    clamp_upper=0.01
    clip_coeff=0.1
    sigma=2.0
    num_epochs=50
    enable_privacy=0




opt = Class_opt()
opt.target_variable='churn'
opt.categorical = True
opt.model='dp-wgan'
opt.target_epsilon=90
opt.target_delta=0.1
opt.batch_size=64
opt.micro_batch_size=8
opt.clamp_lower=0.01
opt.clamp_upper=0.01
opt.clip_coeff=0.1
opt.sigma=0.8
opt.num_epochs=100
opt.enable_privacy=0

opt.downstream_task="classification"

In [ ]:
data_columns = [col for col in train.columns if col != opt.target_variable]

In [ ]:
class_ratios = train[opt.target_variable].sort_values().groupby(train[opt.target_variable]).size().values/train.shape[0]
class_ratios

array([0.50581429, 0.49418571])

In [ ]:
X_train = np.nan_to_num(train.drop([opt.target_variable], axis=1).values)
y_train = np.nan_to_num(train[opt.target_variable].values)
X_test = np.nan_to_num(test.drop([opt.target_variable], axis=1).values)
y_test = np.nan_to_num(test[opt.target_variable].values)

In [ ]:
#Normalized the data
X_train = expit(X_train)
X_test = expit(X_test)

In [ ]:
input_dim = X_train.shape[1]
z_dim = int(input_dim / 4 + 1) if input_dim % 4 == 0 else int(input_dim / 4)

Hyperparams = collections.namedtuple('Hyperarams','batch_size micro_batch_size clamp_lower clamp_upper clip_coeff sigma class_ratios lr num_epochs')
Hyperparams.__new__.__defaults__ = (None, None, None, None, None, None, None, None, None)

In [ ]:
conditional = (opt.downstream_task == "classification")
model = dp_wgan.DP_WGAN(input_dim, z_dim, opt.target_epsilon, opt.target_delta, conditional)
model.train(X_train, y_train, Hyperparams(batch_size=opt.batch_size, micro_batch_size=opt.micro_batch_size,
                                              clamp_lower=opt.clamp_lower, clamp_upper=opt.clamp_upper,
                                              clip_coeff=opt.clip_coeff, sigma=opt.sigma, class_ratios=class_ratios, lr=
                                              5e-5, num_epochs=opt.num_epochs), private=opt.enable_privacy)

torch.save(model, 'drive/My Drive/checkpoint_GAN_MWOE_churn_2-70percent.pth')

Epoch : 1 Loss D real :  0.3820572700371687 Loss D fake :  0.015844269001305013 Loss G :  0.015155301665575174 Epsilon spent :  0
Epoch : 2 Loss D real :  0.38445937604780483 Loss D fake :  0.012447103213268675 Loss G :  0.011694101890798428 Epsilon spent :  0
Epoch : 3 Loss D real :  0.38621509937047166 Loss D fake :  0.010009217077557721 Loss G :  0.010118060452458392 Epsilon spent :  0
Epoch : 4 Loss D real :  0.38880828724207667 Loss D fake :  0.01004072108357695 Loss G :  0.01 Epsilon spent :  0
Epoch : 5 Loss D real :  0.388861550144461 Loss D fake :  0.01 Loss G :  0.01 Epsilon spent :  0
Epoch : 6 Loss D real :  0.38887544657523865 Loss D fake :  0.01 Loss G :  0.01 Epsilon spent :  0
Epoch : 7 Loss D real :  0.3900990794590003 Loss D fake :  0.01 Loss G :  0.01 Epsilon spent :  0
Epoch : 8 Loss D real :  0.3886763206323295 Loss D fake :  0.01 Loss G :  0.01 Epsilon spent :  0
Epoch : 9 Loss D real :  0.39012158927812246 Loss D fake :  0.01 Loss G :  0.01 Epsilon spent :  0
Epo

In [ ]:
if opt.model == 'imle' or opt.model == 'dp-wgan' or opt.model == 'pate-gan':
    syn_data = model.generate(X_train.shape[0], class_ratios)
    X_syn, y_syn = syn_data[:, :-1], syn_data[:, -1]

In [ ]:
X_syn_df = pd.DataFrame(data=X_syn, columns=data_columns)
y_syn_df = pd.DataFrame(data=y_syn, columns=[opt.target_variable])

syn_df = pd.concat([X_syn_df, y_syn_df], axis=1)
syn_df.to_csv("drive/My Drive/synthetic_data_GAN_mWOE_2_70percent_training.csv")
test.to_csv("drive/My Drive/Original_data_GAN_mWOE_2_30percent_test.csv")
print("Saved synthetic data at : ", "drive/My Drive/synthetic_data_GAN_mWOE_2_70percent.csv")

Saved synthetic data at :  drive/My Drive/synthetic_data_GAN_mWOE_2_70percent.csv


In [ ]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}

    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping


        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)

        return X_encoded

In [ ]:
syn_df=pd.read_csv('drive/My Drive/synthetic_data_GAN_mWOE_2_70percent_training.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head()

,Unnamed: 0,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,...,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,new_cell,crclscod,asl_flag,prizm_social_one,area,dualband,refurb_new,hnd_webcap,ownrent,dwlltype,marital,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd
0,0.007933,-0.228536,0.007933,-1.015310,0.487185,-0.471970,0.244246,0.244246,0.244246,0.744749,0.487185,-0.228536,0.007933,0.487185,0.487185,0.744749,-0.228536,0.007933,-0.471970,0.487185,-0.471970,0.244246,0.007933,0.487185,0.744749,-0.228536,0.744749,0.487185,0.244246,0.007933,-0.471970,0.744749,-0.228536,0.244246,0.007933,0.744749,0.487185,-0.228536,0.744749,-1.015310,...,0.007933,-0.471970,0.744749,0.487185,0.007933,-0.228536,0.244246,-0.228536,0.744749,0.244246,0.007933,0.244246,0.487185,0.244246,-0.228536,0.487185,0.244246,0.244246,0.007933,0.487185,0.487185,-0.228536,0.007933,-0.228536,-0.228536,0.244246,0.007933,-0.228536,0.007933,0.244246,0.487185,-0.471970,-0.228536,0.744749,0.487185,-0.228536,0.007933,-0.228536,-0.228536,0.007933
1,0.007933,-0.228536,0.007933,-0.228536,0.487185,-0.730463,0.007933,-1.015310,0.007933,-0.228536,-0.471970,-0.228536,0.007933,-0.471970,-0.228536,-0.471970,-0.730463,-0.228536,-0.228536,0.244246,0.007933,-0.228536,0.244246,0.244246,0.007933,-0.471970,-0.471970,-0.471970,-0.471970,0.244246,-0.471970,-1.015310,0.244246,-0.471970,-0.228536,-0.730463,-0.730463,0.487185,0.007933,0.244246,...,0.007933,-0.730463,-0.471970,-0.228536,0.244246,0.487185,0.007933,-0.730463,0.007933,0.244246,-0.228536,-0.730463,-0.471970,-0.471970,0.244246,0.244246,-0.730463,0.007933,-0.228536,-0.228536,0.244246,0.000000,-1.015310,0.487185,-0.471970,-0.228536,-0.471970,0.007933,-0.228536,-0.228536,-0.730463,0.007933,0.744749,0.244246,-0.730463,0.487185,-0.228536,0.007933,-0.471970,-0.228536
2,0.007933,-0.471970,-0.730463,-0.471970,0.007933,0.244246,0.244246,-0.228536,-0.228536,0.487185,-0.228536,-1.015310,-0.471970,0.007933,0.244246,0.244246,-0.228536,-0.471970,0.007933,0.244246,-0.471970,-0.228536,0.244246,-0.471970,0.244246,0.007933,0.744749,-0.471970,0.244246,-0.228536,-0.228536,0.244246,0.007933,-0.471970,0.007933,-1.015310,0.244246,-0.228536,0.244246,0.007933,...,0.007933,-0.730463,0.007933,0.487185,-0.471970,0.007933,0.007933,0.007933,0.487185,0.007933,-0.730463,0.244246,0.487185,0.007933,-0.730463,-0.471970,0.007933,-1.015310,0.244246,-0.471970,0.007933,-0.471970,0.007933,-0.471970,0.244246,-0.228536,-0.228536,-0.228536,0.007933,-0.471970,-0.228536,0.487185,-0.228536,-0.228536,-0.228536,0.244246,-0.471970,0.244246,-0.471970,-0.228536
3,0.007933,0.007933,0.244246,-0.471970,-0.471970,-0.228536,0.007933,-0.471970,0.244246,-0.471970,-0.228536,-0.228536,0.744749,0.487185,-0.730463,-0.228536,0.244246,0.007933,-0.471970,0.487185,-0.471970,-0.228536,0.007933,0.244246,0.244246,-0.730463,-0.730463,0.244246,0.007933,-0.228536,0.244246,0.487185,-0.471970,0.244246,0.007933,0.744749,0.007933,-0.228536,-0.228536,0.244246,...,-0.471970,0.007933,-0.471970,0.244246,-0.471970,-0.471970,-1.015310,-0.228536,-0.471970,0.007933,-0.730463,-0.228536,0.487185,0.244246,0.007933,0.007933,-0.471970,-0.471970,0.244246,-0.228536,-0.471970,0.007933,-0.471970,0.007933,0.487185,0.487185,-0.471970,-0.730463,0.007933,-0.228536,0.007933,0.744749,-0.730463,-0.730463,-0.228536,-0.471970,0.007933,-0.471970,-0.730463,0.007933
4,0.007933,-0.228536,-0.228536,0.487185,0.007933,-0.471970,0.487185,0.007933,0.007

In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn)


cv_method = RepeatedKFold(n_splits=10,
                          n_repeats=3,
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn_df, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_syn_df, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

35184 52 48 34716
pod:  0.9986192613047981
pof:  0.0014757634237711431
AUC:  0.9985717489405134
accuracy:  0.9985714285714286


In [ ]:
train_df=pd.read_csv('drive/My Drive/Original_data_GAN_mWOE_2_30percent_test.csv')
y_test=train_df['churn']
X_test=train_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head()

,Unnamed: 0,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,...,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,new_cell,crclscod,asl_flag,prizm_social_one,area,dualband,refurb_new,hnd_webcap,ownrent,dwlltype,marital,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd
0,1.437683,0.438523,0.313311,-0.212614,0.023354,0.272117,0.003481,0.177944,0.012381,0.011871,0.065856,0.002696,-0.141508,0.003121,0.418318,0.002758,0.194593,0.012919,-0.033138,0.000771,-0.144748,0.011646,0.065610,0.066488,0.073968,0.233981,0.035462,0.170147,0.016879,-0.306999,-0.406773,0.639069,-0.187007,0.513857,-0.375624,0.008716,0.639069,0.009909,0.283367,0.005483,...,-0.306999,0.313311,1.039462,-0.156376,-0.166814,0.513857,0.178306,0.069988,0.292933,0.066103,0.059352,-0.000358,-0.005735,0.050631,-0.029895,-0.072531,-0.017024,0.002224,0.617782,0.013013,0.095777,0.0474,-0.014155,0.041675,0.146899,-0.027948,0.236921,-0.036852,-0.058392,-0.024435,-0.033376,-0.045371,-0.056840,0.079190,0.004204,0.001431,0.005313,0.004207,0.005265,-0.028516
1,-0.306999,-0.163961,-0.632909,-0.301970,0.023354,0.003890,0.589992,0.537637,0.012381,0.011871,0.003404,0.002696,0.103595,0.003121,-0.087403,0.002758,0.513857,0.012919,-0.033138,0.000771,-0.966045,0.011646,-0.204008,-0.060905,-0.734299,0.100242,-0.064649,0.003179,0.016879,-0.306999,-0.145795,-0.632909,0.035347,-0.632909,0.003674,0.008716,-0.163961,0.009909,-0.543300,0.005483,...,-0.632909,-0.995608,0.313311,-0.798904,0.036678,-0.567158,-0.538583,-0.052771,-0.343276,0.066103,0.059352,-0.000358,-0.005735,0.096934,0.111627,0.089167,0.029305,0.002224,-0.085955,0.002447,0.033759,0.0474,0.040969,-0.106260,-0.025757,-0.027948,-0.084958,0.065858,0.076933,0.088142,-0.033376,0.074457,0.071067,-0.055289,0.004204,0.001431,0.005313,0.004207,0.005265,0.075098
2,0.003179,0.313311,0.514298,0.332905,0.023354,-0.047286,-0.041586,-0.054922,0.012381,-0.264972,-0.039666,0.002696,-0.003543,0.003121,-0.012362,0.002758,0.324752,0.012919,-0.124705,0.000771,-0.084342,0.011646,0.046233,-0.005443,-0.156709,-0.148454,0.035462,-1.432487,0.016879,0.370692,-0.061077,0.253873,0.124656,0.131346,0.230270,0.008716,-0.139858,0.009909,-0.068082,0.005483,...,-0.139858,-0.163961,-0.372643,-0.220783,-0.186821,0.003880,0.136455,-0.059028,-0.075674,-0.088213,-0.174997,0.001590,-0.005735,-0.216129,-0.029895,0.012262,-0.047168,0.002224,-0.363980,0.002447,-0.075056,0.0474,-0.014155,-0.161642,-0.339789,-0.027948,-0.084958,-0.036852,-0.058392,-0.095783,-0.033376,0.074457,-0.056840,0.080604,0.004204,0.001431,0.005313,0.004207,0.005265,-0.028516
3,0.313311,-0.166884,-0.139858,-0.212614,0.023354,-0.047286,-0.041586,-0.054922,0.012381,0.011871,0.084238,0.002696,0.060359,0.003121,-0.175195,0.002758,0.340302,0.012919,0.066611,0.000771,0.328146,0.011646,0.065610,0.066488,0.073968,0.043614,0.035462,0.003179,0.016879,1.101565,0.095284,0.313311,0.124656,0.131346,0.288508,0.008716,-0.507795,0.009909,0.243307,0.005483,...,1.101093,0.003179,-0.159751,-0.163015,-0.071297,0.681730,-0.086247,0.122587,0.292933,0.066103,0.059352,-0.000358,-0.005735,0.096934,-0.029523,-0.072531,0.029305,0.002224,0.545013,0.002447,0.095777,0.0474,0.040969,-0.161642,0.146899,-0.027948,0.236921,0.065858,-0.058392,0.088142,-0.033376,-0.015970,-0.056840,-0.015729,0.004204,0.001431,0.005313,0.004207,0.005265,0.075098
4,-0.306999,0.003179,0.271556,-0.632909,0.023354,0.049976,-0.727461,-0.798904,0.012381,0.011871,0.809331,0.002696,-0.7

In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10,
                          n_repeats=3,
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

12591 2440 1550 13419
pod:  0.8964526688489545
pof:  0.16233118222340495
AUC:  0.8670607433127747
recall:  0.8964526688489545
precision:  0.8461441452802825
F1-Score:  0.8705722070844687
accuracy:  0.867


In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

12485 2546 1811 13158
pod:  0.8790166343777139
pof:  0.16938327456589714
AUC:  0.8548166799059084
recall:  0.8790166343777139
precision:  0.8378757004584819
F1-Score:  0.857953248785577
accuracy:  0.8547666666666667


In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

13591 1440 7021 7948
pod:  0.5309639922506514
pof:  0.09580200918102588
AUC:  0.7175809915348127
recall:  0.5309639922506514
precision:  0.8466126970600767
F1-Score:  0.6526255285954757
accuracy:  0.7179666666666666


In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

12130 2901 1789 13180
pod:  0.880486338432761
pof:  0.1930011309959417
AUC:  0.8437426037184097
recall:  0.880486338432761
precision:  0.8196007710963249
F1-Score:  0.8489533011272142
accuracy:  0.8436666666666667


In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

11499 3532 8505 6464
pod:  0.43182577326474714
pof:  0.23498103918568292
AUC:  0.5984223670395321
recall:  0.43182577326474714
precision:  0.6466586634653861
F1-Score:  0.5178449829761665
accuracy:  0.5987666666666667


In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

14328 703 10807 4162
pod:  0.2780412853230009
pof:  0.046770008648792494
AUC:  0.6156356383371041
recall:  0.2780412853230009
precision:  0.8554984583761562
F1-Score:  0.41968337198749617
accuracy:  0.6163333333333333


In [ ]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0)

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


14022 1009 10346 4623
pod:  0.308838265749215
pof:  0.06712793559976049
AUC:  0.6208551650747273
recall:  0.308838265749215
precision:  0.8208451704545454
F1-Score:  0.44881316440949465
accuracy:  0.6215


In [ ]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Epoch 1/30
1750/1750 [==============================] - 8s 5ms/step - loss: 0.0187 - accuracy: 0.9937 - val_loss: 0.0073 - val_accuracy: 0.9977
Epoch 2/30
1750/1750 [==============================] - 8s 4ms/step - loss: 0.0043 - accuracy: 0.9986 - val_loss: 0.0059 - val_accuracy: 0.9984
Epoch 3/30
1750/1750 [==============================] - 8s 4ms/step - loss: 0.0031 - accuracy: 0.9989 - val_loss: 0.0069 - val_accuracy: 0.9979
Epoch 4/30
1750/1750 [==============================] - 7s 4ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 0.0100 - val_accuracy: 0.9984
Epoch 5/30
1750/1750 [==============================] - 7s 4ms/step - loss: 0.0022 - accuracy: 0.9992 - val_loss: 0.0078 - val_accuracy: 0.9981
12463 2568 1881 13088
pod:  0.8743403032934732
pof:  0.17084691637282948
AUC:  0.8517466934603218
recall:  0.8743403032934732
precision:  0.8359734287174246
F1-Score:  0.854726530612245
accuracy:  0.8517


In [ ]:
#RNN
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
max_features = 10000 # number of words to consider as features
import numpy as np
#create model
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
train_en_x=X_encoded.copy(deep=True)
test_en_y=X_encoded_test.copy(deep=True)
train_en_x[train_en_x < 0]=0
test_en_y[test_en_y < 0]=0
model_history = model.fit(train_en_x, y_syn_df, epochs=5, batch_size=128, validation_split=0.2)

y2_pred = model.predict(test_en_y)
y22_pred=y2_pred.round()

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


Epoch 1/5
438/438 [==============================] - 37s 78ms/step - loss: 0.4916 - acc: 0.7562 - val_loss: 0.4259 - val_acc: 0.8229
Epoch 2/5
438/438 [==============================] - 33s 76ms/step - loss: 0.4116 - acc: 0.8184 - val_loss: 0.4016 - val_acc: 0.8229
Epoch 3/5
438/438 [==============================] - 33s 76ms/step - loss: 0.4072 - acc: 0.8206 - val_loss: 0.4028 - val_acc: 0.8229
Epoch 4/5
438/438 [==============================] - 33s 76ms/step - loss: 0.4039 - acc: 0.8226 - val_loss: 0.4167 - val_acc: 0.8229
Epoch 5/5
438/438 [==============================] - 34s 77ms/step - loss: 0.4048 - acc: 0.8194 - val_loss: 0.4019 - val_acc: 0.8229
11103 3928 3736 11233
pod:  0.750417529561093
pof:  0.261326591710465
AUC:  0.7445454689253139
recall:  0.750417529561093
precision:  0.7409141877184883
F1-Score:  0.7456355791569864
accuracy:  0.7445333333333334
